# Feature Engineering Overview
In our previous tutorials, we only brushed upon features and how to handle them. In this overview we'll take a practical approach to learning about feature engineering. The things we will focus on are:
- Develop a baseline model for comparing performances on models with more/different features.
- Encode categorical features so model can make better use of the information.
- Generate new features to provide more information for the model.
- Select specific features to reduce overfitting and increase prediction speed.

In the main exercise, we'll be using the 'TalkingDataAdTracking' kaggle competition dataset. The goal of this dataset is to predict if a user will download an app after clicking through an ad. For learning purposes we'll drop 99% of negative records (negative meaning the app wasn't downloaded) to make the target more balanced.

[1. Baseline Model](#baseline)  
[2. Categorical Encodings](#encode)  
[3. Feature Generation](#generate)  
[4. Feature Selection](#select)

#### Notes for self:
- **kst** is the raw imported data
- **kst_data** is timestamp encoded
- **base_data** is timestamp and label encoded

<a id='baseline'></a>
## 1. Baseline Model
In this overview we'll be using Kickstarter data.

### Kickstarter Warmup (review)

In [ ]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
from termcolor import colored, cprint # colored prints
from my_modules import data_imports as data
cprint('Imported Modules', 'green')

In [ ]:
kst_data = data.import_kickstarter_2018_data()
kst_data.head(10)


Looking at this data, let's try to predict whether or not a Kickstarter project will succeed or not. To build teach our model, we can use the *state* column as our outcome. To predict this outcome, we can use features such as category, currency, funding goal, country, and when it was launched.

### Preparing target column
First, let's look at project states and convert them into something we can use as targets in a model. Remember that model's don't like to work with strings, and our outcome data is categorical.

In [ ]:
# pd.unique(kst_data.state)
# kst_data.groupby('state')['ID'].count()
kst_data.groupby('state')['ID'].nunique()

So we see that our dataset has 6 unique states, with mostly failed and successful outcomes.

Since our priority in this quick review is not data cleaning, we'll just go a long with this simple cleansing:
- Drop projects that are "live"
- Counting successful as ```outcome = 1```
- Combining all other states as ```outcome = 0```

In [ ]:
# Drop live projects
kst_data = kst_data.query('state != "live"')

# Add the 'outcome' column with "successful == 1", everything else 0
kst_data = kst_data.assign(outcome=(kst_data['state'] == 'successful').astype(int))
kst_data.head()

### Converting Timestamps
Now that we have our outcome all setup and ready, it's time to handle dates. Let's convert the *launched* feature into something more categorical that our model can understand. We imported both *deadline* and *launched* as python Timestamp objects, so we can use the ```.dt.``` attribute on the timestamp column to get the times. 

In [ ]:
# Note that this below syntax doesn't work on a single Timestamp object. dt must be used on a column
#    kst_data['launched'][0].dt

kst_data = kst_data.assign(
    hour=kst_data.launched.dt.hour,
    day=kst_data.launched.dt.day,
    month=kst_data.launched.dt.month,
    year=kst_data.launched.dt.year
)
kst_data.head()

### Prepping categorical variables
Now we that both our outcome AND timestamp data setup, it's time to get our other categorical variables in check! For our model, we'll be using *category*, *currency*, and *country*, which all need to be converted into integer representations. We'll use scikit-learn's ```LabelEncoder``` for this.

In [ ]:
# print(kst_data.groupby('category')['ID'].nunique())
# print(kst_data.groupby('currency')['ID'].nunique())
# print(kst_data.groupby('country')['ID'].nunique())

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_features = ['category', 'currency', 'country']
encoder = LabelEncoder()

# Apply the label encoder to each column
encoded_kst = kst_data[cat_features].apply(encoder.fit_transform)
encoded_kst.head(10)

Great! Now let's gather all of the columns we're using for this model into a new, clean little dataframe. Because our original dataframe and our encoded dataframe have the same index, we can ```join``` them together easily.

In [ ]:
# kst_data has our hand-encoded hour, day, month, year, and outcome while 'encoded' has the labelencoded data. They both have the same index, so join join join!
base_data = data = kst_data[['goal', 'hour', 'day', 'month', 'year', 'outcome']].join(encoded_kst)
data.head()

### Creating training, validation, and test splits
Ain't our data pretty? Now that's it's ready to go, it's time to split up our data into training, validation and test splits! Since this is just a quick review, let's take a simple approach just use slices of our data. We'll use 10% of the data as validation, 10% for testing, and 80% for training.

**Note:** For python beginners (like me), there are extra steps/comments below to explain the indexing in the end

In [ ]:
valid_fraction = 0.1
valid_size = int(len(data) * valid_fraction)

print("len of data: {}".format(len(data)))
print("valid size: {}".format(valid_size))
# The below indexing is a little confusing, so let's analyze it
# We need 80% of the set for training
print("80% of the dataset: {}".format(round(len(data) * 0.8)))
# This comes out to 300689, which is a difference of...
print( "Full data size - 80% data size: {}".format(round(len(data) - (len(data) * 0.8))))
# 75172! ... hmmmmmm now why are we using 2 * valid_size below?
print("valid_size doubled: {}".format(valid_size * 2))
# They're the same!!! valid_size * 2 === the difference from above!
# Oh ya.... valid_fraction = 0.1, so 100% - (10% * 2) = 80% .... I see

# Remember that python uses the colon as [start:end] accessor, and using negatives gives us the opposite, so [:1] is from start to the first element, and [:-1] is from the start, to the end-1

# start : end - (valid_size * 2), [0 : 375862 - 75173]
train = data[:-2 * valid_size]
# end - (valid_size*2) : end - valid_size, [375862 - 75173 : 375862 - 37586]
valid = data[-2 * valid_size:-valid_size]
# end - valid_size : end, 375862 - 37586 : 375826]
test = data[-valid_size:]

print("Length of train/valid/test: {}/{}/{}".format(len(train), len(valid), len(test)))

In general, we want to be careful that each data set has the same proportion of the target classes (keep spliced data balanced). Let's print out the fraction of successful outcomes from each dataset to confirm:

In [ ]:
# In the above block, we used traditional Python3 string formatting.
# Below, we use the new 3.6 F-strings!
# The below statement would most similary equal:
#   print("Outcome fraction = {:.4f}".format(each.outcome.mean()))

for each in [train, valid, test]:
    print(f"Outcome fraction = {each.outcome.mean():.4f}")

As we can see, each splice has around 35% true outcomes, likely because the data was well randomized beforehand. If this weren't the case, we could have used a helpful sklearn method: ```sklearn.model_selection.StratifiedShuffleSplit```.

### Training a LightGBM model
In previous examples, we used Random Regression Trees and XGBoost. This time around, we'll be using a *LightGBM* model. This is a tree-based model that typically provides the best performance, even compared to XGBoost. This time around our model won't be very optimized (as this is just a quick review) but we'll still see improvement through our feature engineering.

In [ ]:
# if lightgbm can't be found, run the following command (if using conda)
#   conda install -c conda-forge lightgbm
import lightgbm as lgb

feature_cols = train.columns.drop('outcome')

# Read the docs on lightgbm for more info on the parameters
dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])

param = {'num_leaves' : 64, 'objective':'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10, verbose_eval=False)
print(colored("Good to go!", 'green'))

### Making predictions & evaluating the model
Now that we got the model all setup and trained, let's make some predictions on the test set with the model and see how it performs.

In [ ]:
from sklearn import metrics
ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['outcome'], ypred)

print(f"Test AUC score: {score}")

And that's it for the basic baseline! Now we can move on engineering our features further.

<a id='encode'></a>
## 2. Categorical Encodings
Now that we have a nice lil baseline model, it's time to engineer it a little more. In a previous lesson, Intermediate Machine Learning, we learned about one-hot encoding and in this overview we used basic label coding above. Now we'll learn about a few more encodings, specifically:
- Count Encoding
- Target Encoding
- Singular Value Decomposition

In [ ]:
# Let's define some helper function for testing our encodings. It'll be based off of lightgbm and data prep from above
#  - Helper functions defined in 'feature_engineering.py'
from my_modules import feature_engineering as fe
train, valid, _ = fe.get_kickstarter_splits(base_data)
bst = fe.train_kickstarter_model(train, valid)

### Count Encoding
Count encoding replaces each categorical value with the number of times it appears in the dataset. For this encoding, we'll use *categorical-encodings* package, specifically ```CountEncoder```. This encoder and the others in *categorical-encodings* work like scikit-learn transformers with ```.fit``` and ```.transform``` methods.


In [ ]:
# category_encoders conda install:
#   $ conda install -c conda-forge category_encoders
import category_encoders  as ce
cat_features = ['category', 'currency', 'country']
count_enc = ce.CountEncoder()
# kst_data from above, after timestamp encoding, pre basic encoding
count_encoded = count_enc.fit_transform(kst_data[cat_features])

category_data = base_data.join(count_encoded.add_suffix("_count"))

# Training and testing
train, valid, _ = fe.get_kickstarter_splits(category_data)
bst = fe.train_kickstarter_model(train, valid)

A slight increase from 0.7467 -> 0.7486

### Target Encoding
Target encoding replaces a categorical value with the average value of the target for that value of the feature. For example, given the country value "CA", we would calculate the average outcome for all the rows with ```country == 'CA'```. This is often blended with the target probability over the entire dataset to reduce the variance of values with few occurences.

This technique uses the targets to create new features. So including the validation or test data in the target encodings would be a form of target leakage. We should only learn the target encodings from the training dataset only and apply it to the other datasets.

Much like ```CountEncoder```, we'll use ```TargetEncoder``` from *category_encoders*.

In [ ]:
cat_features = ['category', 'currency', 'country']

target_enc = ce.TargetEncoder(cols=cat_features)

train, valid, _ = fe.get_kickstarter_splits(category_data)

target_enc.fit(train[cat_features], train['outcome'])

train = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
valid = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_target'))

train.head()
bst = fe.train_kickstarter_model(train, valid)

Adding target encoding on top of count encoding has given us another increase, 
0.7486 -> 0.7491

### CatBoost Encoding
Finally we'll look at CatBoost encoding. CatBoost is similar to target encoding in that it's based on the target probability for a given value. However, with CatBoost, for each row, the target probability is calculated only from the rows before it.

In [ ]:
cat_features = ['category', 'currency', 'country']
cat_boost = ce.CatBoostEncoder(cols=cat_features)

train, valid, _ = fe.get_kickstarter_splits(category_data)
cat_boost.fit(train[cat_features], train['outcome'])

train = train.join(cat_boost.transform(train[cat_features]).add_suffix("_cb"))
valid = valid.join(cat_boost.transform(valid[cat_features]).add_suffix("_cb"))

bst = fe.train_kickstarter_model(train, valid)

With our current model, CatBoost only gave us a 0.0001 improvement over target encoding.

<a id='generate'></a>
## 3. Feature Generation
Creating new features from raw data is one of the best ways we can improve our models. For example, we can calculate the number of total projects in the last week and the duration of the fundraising period. Of course these new features will be different for all datasets, so it takes a bit of creativity. 

### Interactions
One of the easiest ways to create new features is by combining categorical variables.
For example, we combine the country *CA* and category *Music* and create a new value *CA_Music*. This new categorical feature can provide info about correlations between variables. This type of feature is typically called an interaction. In general we build interaction features from all pairs of categorical features. 

Be sure label encode the new combined feature!

In [ ]:
# We use kst_data to get the interactions, and then labelencode them and append to base_data
interactions = kst_data['category'] + "_" + kst_data['country']
cprint('Interactions Head: ', 'cyan')
print(interactions.head(5))

label_enc = LabelEncoder()
interaction_data = base_data.assign(category_country=label_enc.fit_transform(interactions))
cprint('Appended Interaction Data', 'cyan')
interaction_data.head(5)
  

## Counting number of projects
Next we'll see how to the number of projects launched in the preceeding week for *each* record. We'll use the ```.rolling()``` method on a series with the *launched* column as the index. We'll first create the series using ```kst_data.launched``` as the index and ```ks_data.index``` as the values, then sort the times. Using a time series as the index allows us to define the rolling window size in terms of hours, days, weeks, etc.

In [ ]:
# First create a Series with a timestamp index
launched = pd.Series(kst_data.index, index=kst_data.launched, name="count_7_days").sort_index()
launched.head(15)


`One thing to note before continuing is that a few of the projects don't have valid start dates. For now we'll just ignore this, but in a real example these type of things should be cleansed out.

Now, with a timeseries as our index, we can use ```.rolling()``` to select time periods as the window. For example, ```launched.rolling('7d')``` creates a rolling window that contains all the data in the previous 7 days. The window contains the current record, so if we want to count all the *previous* projects but not the current one, we'll need to subtract 1. We'll plot the results to make sure it all looks right.

In [ ]:
# help(launched.rolling)
count_7_days = launched.rolling('7d').count() - 1
print(count_7_days.head(20))

import matplotlib.pyplot as plt
# plot the count ignoring the first 7 records
plt.plot(count_7_days[7:])
plt.title("Competitions in the last 7 days relative to current project")

Very interesting spike there in 2014...

Anyways, now that we have the counts of projects that occured in the last 7 days relative to each project, we must adjust the index so we can join this new data with the training data.

In [ ]:
# unlike a DataFrame, series don't have a set_index() function
print(launched)
count_7_days.index = launched.values
count_7_days = count_7_days.reindex(kst_data.index)
cprint('Reindexed count_7_days', 'cyan')
print(count_7_days.head(10))

# Now join this beautiful new feature onto the training data
cprint('New base_data', 'cyan')
base_data = base_data.join(count_7_days)

### Time since the last project in the same category
Do projects in the same category compete for donors? If we're trying to fund a video game, and another game project in the same category was just launched, we might not get as much money, right? We can capture this by calculating the time since the last project launch in the same category.

A handy method for performing operations within groups is to use the ```.groupby()``` then ```transform()``` functions. The ```transform()``` method takes a function then passes a series or dataframe to the function for each group. This returns a dataframe with the same indices as the original dataframe. In our case, we'll perform a groupby on *category* and use transform to calculate the time differences for each category.

In [ ]:
def time_since_last_project(series):
    # return the time in hours
    # diff calculates the difference of a series element compared with another (default previous)
    return series.diff().dt.total_seconds() / 3600.
df = kst_data[['category', 'launched']].sort_values('launched')
timedeltas = df.groupby('category').transform(time_since_last_project)
timedeltas.head(25)


We get ```NaN``` values for projects that are the first in their category. We'll need to fill these in with something like the mean or median. We'll also need to reset the index again so we can merge into our training data.

In [ ]:
# final time since last project
timedeltas = timedeltas.fillna(timedeltas.median()).reindex(base_data.index)
cprint('Fixed timedeltas', 'cyan')
print(timedeltas.head(10))
cprint('New base_data', 'cyan')
base_data = base_data.join(timedeltas)
base_data.head(10)

### Transforming numerical features
The distribution of the values in *goal* shows that most projects have goals less than 5000 USD. However, there is a long tail of goals going up to $100,000. Some models work better when features are normally distributed, so it might help to transform the goal values. Common choices for this are the sqrt and natural logarithm. These transformations can also help constrain outliers.

Let's transform the goal feature using the sqrt and log functions and graph it to show the changes.

In [ ]:
plt.hist(kst_data.goal, range=(0, 100000), bins=50)
plt.title('Goal')

In [ ]:
import numpy as np
plt.hist(np.sqrt(kst_data.goal), range=(0,400), bins=50)
plt.title('Sqrt(Goal)')

In [ ]:
plt.hist(np.log(kst_data.goal), range=(0,25), bins=50)
plt.title('Log(Goal)')

The log transformation won't actually help our model since tree-based models are scale invariant (regardless of how much we zoom in or out, it will all look the same). However, this would help if we had a linear model or neural network.

Other transformations include squares and other powers, exponentials, etc. These might help the model discriminate, like ther kernel trick for SVMs. It takes a bit of experimentation to see what works. One method is to create a bunch of new features and later choose the the best ones with feature selection algorithms.

<a id='select'></a>
## 4. Feature Selection
Often we'll have hundreds or thousands of features after various encodings and feature generations. This can lead to two main problems. 
First, the more features we have, the more likely we will overfit to the training and validation sets. This will cause our model to perform worse at generalizing to new data.

Secondly, the more features we have, the longer it will take to train our model and optimize hyperparameters. Also, when building user-facing products, we'll want to make inference as fast as possible. Using fewer features can speed up inference at the cost of predictive performance.

To help with these issues, we'll want to use feature selection techniques to keep the most informative features for our model.

Let's get out data for this section setup.

In [ ]:
from itertools import combinations
# To follow along with the kaggle tutorial, let's re-encode and do a bit of review
# Timestamped data
# kst_data

# Label Encoding
cat_features = ['category', 'currency', 'country']
lab_encoder = LabelEncoder()
encoded = kst_data[cat_features].apply(lab_encoder.fit_transform)

data_cols = ['goal', 'hour', 'day', 'month', 'year', 'outcome']
baseline = kst_data[data_cols].join(encoded)

# Interactions
interactions = pd.DataFrame(index=kst_data.index)
for col1, col2 in combinations(cat_features, 2):
    new_col_name = '_'.join([col1, col2])
    # convert to strings and combine
    new_values = kst_data[col1].map(str) + "_" + kst_data[col2].map(str)
    interactions[new_col_name] = lab_encoder.fit_transform(new_values)
baseline = baseline.join(interactions)

# last 7 days of projects in categories
launched = pd.Series(kst_data.index, index=kst_data.launched, name='count_7_days').sort_index()
count_7_days = launched.rolling('7d').count() - 1
count_7_days.index = launched.values
count_7_days = count_7_days.reindex(kst_data.index)

baseline = baseline.join(count_7_days)

# time since last project
df = kst_data[['category', 'launched']].sort_values('launched')
timedelt = df.groupby('category').transform(time_since_last_project)
timedelt = timedelt.fillna(timedelt.max())

baseline = baseline.join(timedelt.rename({'launched':'time_since_last_project'}, axis=1))
baseline

### Univariate Feature Selection
The simplest and fastest methods are based on univariate statistical tests. For each feature, we measure how strongly the target depends on the feature using a statistical test like x<sup>2</sup> or ANOVA.

From the scikit-learn feature selection module, ```feature_selection.SelectKBest``` returns the K best features given some scoring function. For our classification problem, the module provides three different scoring functions: x<sup>2</sup>, ANOVA F-value, and the mutual information score. The F-value measures the linear dependency between the feature variable and the target. This means the score might underestimate the relation between a feature and the target if the relationship is nonlinear. The mutual information score is nonparametric and so can capture nonlinear relationships.

With ```SelectKBest```, we define the number of features to keep, based on the score from the scoring function. Using ```.fit_transform(features, target)``` we get back an array with only the selected features.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

feature_cols = baseline.columns.drop('outcome')

# Keep 5 features
selector = SelectKBest(f_classif, k=5)

X_new = selector.fit_transform(baseline[feature_cols], baseline['outcome'])
X_new

The above example is actually an example of data leakage. The statistical tests were calculated using all of the data, including the validation and test sets. This could influence the features we keep, so we should test only on the training set.

In [ ]:
feature_cols = baseline.columns.drop('outcome')
train, valid, _ = fe.get_kickstarter_splits(baseline)

# Keep 5 features
selector = SelectKBest(f_classif, k=5)

X_new = selector.fit_transform(train[feature_cols], train['outcome'])
X_new

And the selected features have changed! Now we have our selected features, but it's only the feature values for the training set. To drop the rejected features from the validation and test sets, we need to figure out which columns in the dataset were kept with ```SelectKBest```. To do this, we can use ```.inverse_transform``` to get back an array with the shape of the original data.

In [ ]:
# Get back the features we've kept, zero out all other features
selected_features = pd.DataFrame(selector.inverse_transform(X_new),
                                 index=train.index,
                                 columns=feature_cols)
selected_features.head()

This returns a DataFrame with the same index and columns as the training set, but all the dropped columns are filled with zeros. We can find the selected columns by choosing features where the variance is non-zero.

In [ ]:
# Dropped columns have values of all 0s, so var is 0, drop them
selected_columns = selected_features.columns[selected_features.var() != 0]

# Get the valid dataset with the selected features
valid[selected_columns].head()

### L1 regularization
Univariate methods consider only one feature at a time when making a selection decision. Instead, we can make our selection using all of the features by including them in a linear model with L1 regularization. This type of regularization (sometimes called Lasso) penalizes absolute magnitude of the coefficients, as compared to L2 (Ridge) regression which penalizes the square of the coefficients.

As the strength of regularization is increased, features which are less important for predicting the target are set to 0. This allows us to perform feature selection by adjusting the regularization parameter. We choose the parameter by finding the best performance on a hold-out set, or decide ahead of time how many features to keep.

For regression problems, we can use ```sklearn.linear_model.Lasso```, or ```sklearn.linear_model.LogisticRegression``` for classification. These can be used along with ```sklearn.feature_selection.SelectFromModel``` to select the non-zero coefficients. Otherwise, the code is similar to the univariate tests.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

train, valid, _ = fe.get_kickstarter_splits(baseline)

X, y = train[train.columns.drop('outcome')], train['outcome']

# Set the regularization parameter C=1
logistic = LogisticRegression(C=1, penalty="l1", random_state=7).fit(X, y)
model = SelectFromModel(logistic, prefit=True)

X_new = model.transform(X)
X_new

Similar to univariate tests, we get back an array with the selected features. Again, we will want to convert these to a DataFrame so we can ge the selected columns.

In [ ]:
# Get back the kept features as a DataFrame with dropped columns as all 0s
selected_features = pd.DataFrame(model.inverse_transform(X_new),
                                 index=X.index,
                                 columns=X.columns)

# Dropped columns have values of all 0s, keep other columns
selected_columns = selected_features.columns[selected_features.var() != 0]
valid[selected_columns].head()

In this case with the L1 parameter C=1, we're dorpping the *time_since_last_project* column.

In general, feature selection with L1 regularization is more powerful than univariate tests, but it can also be very slow when we have a lot of data and a lot of features. Univariate tests will be much faster on large datasets, but also will likely perform worse.